# TSP

In [11]:
# TSPLIB の XML ファイルの読み込み

import xml.etree.ElementTree as ET
import numpy as np

tree = ET.parse('gr17.xml')   # ファイル名
# gr17 の最短経路は 2085
root = tree.getroot()

ncity = 0  # number of verteces (=cities)
for vertex in root.iter('vertex'):
    ncity += 1

print('number of verteces = ',ncity)

distances = np.zeros((ncity, ncity))

cn = 0
for vertex in root.iter('vertex'):
    for edge in vertex.iter('edge'):
        distances[cn][int(edge.text)] = edge.get('cost')
    cn += 1

number of verteces =  17


# <span style="color: blue; ">QUBO定式化 （PyQUBO）</span>

In [12]:
from pyqubo import Array, Placeholder, Constraint, Binary

# ２値変数 q_{i,j} : 都市 i から都市 j への経路を選択するとき 1
x = Array.create('c', (ncity, ncity), 'BINARY')

In [13]:
# Constraint not to visit more than two cities at the same time.
time_const = 0.0
for i in range(ncity):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    time_const += Constraint((sum(x[i, j] for j in range(ncity)) - 1)**2, label="time{}".format(i))

# Constraint not to visit the same city more than twice.
city_const = 0.0
for j in range(ncity):
    city_const += Constraint((sum(x[i, j] for i in range(ncity)) - 1)**2, label="city{}".format(j))

In [14]:
# distance of route
dist = 0.0
for i in range(ncity):
    for j in range(ncity):
        for k in range(ncity):
            dist += distances[i, j] * x[k, i] * x[(k+1)%ncity, j]

In [15]:
# Construct hamiltonian
A = Placeholder("A")
H = dist + A * (time_const + city_const)

In [16]:
# Compile model
pq_model = H.compile()

In [17]:
# Generate QUBO

# feed_dict = {'A': 0.5}
feed_dict = {'A': np.amax(distances)}
bqm = pq_model.to_bqm(feed_dict=feed_dict)

In [18]:
import neal
import time

st = time.time()  # 時間計測開始
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100, num_sweeps=10000)
et = time.time() - st  # 時間計測修了

# Decode solution
decoded_samples = pq_model.decode_sampleset(sampleset, feed_dict=feed_dict)
best_sample = min(decoded_samples, key=lambda x: x.energy)
num_broken = len(best_sample.constraints(only_broken=True))
print('Number of broken constarint = {}'.format(num_broken))
print('Execution time =',et*1000, '[msec]')

if num_broken == 0:
    city_order = []
    for i in range(ncity):
            for j in range(ncity):
                if best_sample.array('c', (i, j)) == 1:
                    city_order.append(j)

print('path length',sum(
        [distances[city_order[i]][city_order[(i + 1) % ncity]] for i in range(ncity)]
    ))

Number of broken constarint = 0
Execution time = 4867.622137069702 [msec]
path length 2676.0


## ソルバ実行（ループ）

In [ ]:
import csv

# 繰り返し回数
lcount = 10

with open('/home/stomo/Desktop/temp_sa.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['count', 'energy', 'time'])

    for k in range (lcount):
        st = time.time()  # 時間計測開始
        sa = neal.SimulatedAnnealingSampler()
        sampleset = sa.sample(bqm, num_reads=100, num_sweeps=10000)
        et = time.time() - st  # 時間計測修了
        
        # Decode solution
        decoded_samples = pq_model.decode_sampleset(sampleset, feed_dict=feed_dict)
        best_sample = min(decoded_samples, key=lambda x: x.energy)
        num_broken = len(best_sample.constraints(only_broken=True))

        if num_broken != 0:
            raise RuntimeError("Any one of constraints is not satisfied.")

        city_order = []
        for i in range(ncity):
            for j in range(ncity):
                if best_sample.array('c', (i, j)) == 1:
                    city_order.append(j)

        pl = sum(
            [distances[city_order[i]][city_order[(i + 1) % ncity]] for i in range(ncity)]
            )
        
        writer.writerow([k, pl, et*1000])

f.close()

# <span style="color: blue; ">QUBO定式化 （Amplify）</span>

In [19]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
# ２値変数 q_{i,j} : 都市 i から都市 j への経路を選択するとき 1
q = gen.array(ncity, ncity)

### <span style="color: red; ">回転対称性の除去</span>
0番目の都市から開始する

In [ ]:
q[0,0] = 1
for i in range(1,ncity):
    q[0,i] = 0
    q[i,0] = 0

In [20]:
# Cost function
from amplify import sum_poly

cost = sum_poly(
    ncity,
    lambda n: sum_poly(
        ncity,
        lambda i: sum_poly(
            ncity, lambda j: distances[i, j] * q[n, i] * q[(n + 1) % ncity, j]
        ),
    ),
)

In [21]:
# constraint function
from amplify.constraint import one_hot

# 行に対する制約
row_constraints = [one_hot(q[n]) for n in range(ncity)]
# 列に対する制約
col_constraints = [one_hot(q[:, i]) for i in range(ncity)]

### <span style="color: red; ">反転対称性の除去</span>
[ 0 7 1 5 6 3 4 2] と [0 2 4 3 6 5 1 7 ] は反転しているが同一の解。
これを除去する制約を追加する。

$ q_{N-1, i} q_{1,j} = 0 \qquad (i<j)$

In [ ]:
from amplify.constraint import penalty

# 順序に対する制約
pem_constraint = [
    penalty(q[ncity - 1, i] * q[1, j])
    for i in range(ncity)
    for j in range(i + 1, ncity)
]

# 下の制約式変更すること

### 制約

In [22]:
from amplify import sum_poly

# constraints = sum(row_constraints)
constraints = sum(row_constraints) + sum(col_constraints)
# constraints = sum(row_constraints) + sum(col_constraints) + sum(pem_constraint)

constraints *= np.amax(distances)  # 制約条件の強さを設定

### イジングモデル生成

In [23]:
#######################################################
# 確認用
model = cost + constraints

# model.logical_poly
# model.input_constraints
# model.input_constraints[0][0].penalty

### ソルバ生成

In [24]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.token = "rSnIw4H95J8GuAKbvPxzG2PMOCCcFoZD"  #2022-11-27まで有効
client.parameters.timeout = 10000  # タイムアウト10秒（ディフォルト10秒）
client.parameters.outputs.duplicate = False  # エネルギー値が同一の解を異なる解とみなす
client.parameters.outputs.num_outputs = 0   # スピン配列とエネルギー値をすべて出力

solver_am = Solver(client)

### ソルバ実行

In [25]:
from amplify.client import FixstarsClient

# ソルバ result
solver_result = solver_am.solve(model)
if len(solver_result) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

print('# of solutions = ',len(solver_result))

energy = solver_result[0].energy
print('energy = ',energy)

values = solver_result[0].values
q_values = q.decode(values)
# print('q_values = ',q_values)

print('Feasible solution: ', solver_result[0].is_feasible)
print('Execution time =',solver_am.execution_time, '[msec]')

# クライアント result
# client_result = solver_am.client_result
# print('client annealing time', client_result.timing.annealing_time)
# print('client cpu time',client_result.timing.cpu_time)
# print('clent execution num iterations',client_result.execution_parameters.num_iterations)
# print('client unit step',client_result.execution_parameters.num_unit_steps)

# of solutions =  3
energy =  2085.0
Feasible solution:  True
Execution time = 9947 [msec]


### ソルバ実行（ループ）

In [ ]:
import csv

# 繰り返し回数
lcount = 10

with open('/Users/stomo/Desktop/temp_qa.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['count', 'energy', 'time'])

    for k in range (lcount):
        result = solver_am.solve(model)
        if len(result) == 0:
            raise RuntimeError("Any one of constraints is not satisfied.")
        writer.writerow([k, result[0].energy, solver_am.execution_time])

f.close()

values = result[0].values
q_values = q.decode(values)

# <span style="color: blue; ">0-1 整数線形最適化による定式化</span>
訪問順を表す変数（整数値）を導入するので、**混合整数計画問題**となる
## 変数、問題生成

In [26]:
import networkx as nx
from pulp import *
from itertools import combinations

G = nx.complete_graph(ncity)  # 完全グラフの作成
G = nx.to_directed(G)     # 有向グラフ

nodes = list(G.nodes)     # 頂点集合
edges = list(G.edges)     # 辺集合

# 決定変数
xvars = LpVariable.dicts('x', edges, None, None, LpBinary)
# 巡回順変数
wvars = LpVariable.dicts('w', nodes, 1, ncity-1, LpContinuous)

# 問題生成
TSP = LpProblem('TSP', LpMinimize)

## コスト関数
$d_{i,j}$ : 辺 $\{ i,j \}$ のコスト（距離）
$$
 \sum_{i,j \in V} d_{i,j} x_{i,j}
$$

In [27]:
TSP += lpSum([distances[e[0]][e[1]] * xvars[e] for e in edges]), 'Total Cost'

## 1-hot 制約
### 流入辺は1本
$$
 \sum_{j \in V, j \neq i} x_{i,j} = 1 \qquad (i \in V)
$$
### 流出辺は1本
$$
 \sum_{i \in V, i \neq j} x_{i,j} = 1 \qquad (j \in V)
$$

In [28]:
for u in nodes:
    # 流出辺
    TSP += lpSum([xvars[e] for e in edges if e[0] == u]) == 1, 'Flow Conservation (out) %s'%u
    # 流入辺
    TSP += lpSum([xvars[e] for e in edges if e[1] == u]) == 1, 'flow Conservation (in) %s'%u

## 部分順回路除去制約
### MTZ
$w_i$ : 頂点 $i$ の訪問順（ただし、$w_0 = 0$）（頂点 0 は出発点）
$$
 w_i + 1 - (n-1) (1-x_{i,j}) \leq w_j \qquad (i,j = 1, \ldots, n-1, i \neq j)
$$

### 訪問順の上下限
$$
 w_i \geq 2 - x_{0,i} + (n-3) x_{i,0}, \; w_i \leq n-2 + x_{i,0} - (n-3) x_{0,i}
$$

In [29]:
# 制約: 部分順回路除去　(MTZ)
nodeset = set(nodes)
for (u,v) in [(u,v) for u in nodes for v in nodes if u!= 0 and v!=0 and u!=v]:
    TSP += wvars[u] + 1 - (ncity-1)*(1-xvars[(u,v)]) <= wvars[v]
    TSP += 2 - xvars[(0,u)] + (ncity - 3)*xvars[(u,0)] <= wvars[u]
    TSP += ncity - 2 + xvars[(u,0)] - (ncity - 3)*xvars[(0,u)] >= wvars[u]

### 求解

In [30]:
# ソルバスレッド数設定
# solver_lp = COIN_CMD(threads=10, msg=0, timeLimit=10)
solver_lp = COIN_CMD(threads=8, msg=0)
# 求解
status = TSP.solve(solver_lp)

print('Status:', LpStatus[status])
print('Optimal val:', value(TSP.objective))
# for e in edges:
    # if xvars[e].value() > 0.5:
        # print(e,xvars[e].value(), ' cost:', distances[e[0]][e[1]])
print('n:', ncity, ', number of constraints:', len(TSP.constraints))
print('Elapsed time: ', TSP.solutionTime*1000)

Status: Optimal
Optimal val: 2085.0
n: 17 , number of constraints: 754
Elapsed time:  2635.018825531006


### 求解（ループ）

In [ ]:
import csv

# 繰り返し回数
lcount = 10

# ソルバスレッド数設定
solver_lp = COIN_CMD(threads=10, msg=0)

with open('/Users/stomo/Desktop/temp_lp.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['count', 'energy', 'time'])

    for k in range (lcount):
        status = TSP.solve(solver_lp)
        if LpStatus[status] != 'Optimal':
            raise RuntimeError("Any one of constraints is not satisfied.")
        writer.writerow([k, value(TSP.objective), TSP.solutionTime*1000])

f.close()